# Cryptography HelloWorld

In [3]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        self.lproc = Process(target = left, args=(left_end,))
        self.rproc = Process(target = right, args=(right_end,))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

## Receiver e Emitter

In [4]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend

backend = default_backend()
cred_salt = os.urandom(16)
kdf = PBKDF2HMAC(algorithm=hashes.SHA256(), length=32, salt=cred_salt, iterations=100000, backend=backend)

def Receiver(connection):
    pwd = getpass('Reciever password: ')
    kdf.verify(pwd.encode("utf-8"), key)
    payload = conn.recv()
    #TODO: Verify integrity/authenticity and decrypt
    conn.close()

def Emitter(connection):
    pwd = getpass('Emitter password: ')
    key = kdf.derive(pwd.encode("utf-8"))
    payload = 'hello'
    payload
    conn.send(payload)
    conn.close()

## Primitivas Criptográficas

Para executar a aplicação desenvolvida basta invocar o método `manual` da classe `Connection` após criar um objeto desse tipo.

In [2]:
Connection(Receiver, Emitter).manual()

NameError: name 'Connection' is not defined